In [85]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, OutputFixingParser, PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser
import json
from langchain_core.messages import AIMessage
from typing import List

In [5]:
model = ChatOpenAI()

## pydantic output parser

In [17]:
class Actor(BaseModel):
    name: str=Field(description="name of the actor")
    age : int=Field(description="age of the actor")
    @validator("age")
    def age_validator(cls, value):
        if value<0:
            raise ValueError("wrong age!")
        return value
    

In [25]:
parser = PydanticOutputParser(pydantic_object=Actor)
json_data = json.dumps({'name': 'anna', 'age': -1})
parser.parse(json_data)

OutputParserException: Failed to parse Actor from completion {"name": "anna", "age": -1}. Got: 1 validation error for Actor
age
  wrong age! (type=value_error)

In [18]:
parser = PydanticOutputParser(pydantic_object=Actor)

query="""Who is the actor who plays the male lead in the movie Titanic? 
Tell me the age and name of the actor"""

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": query})

Actor(name='Leonardo DiCaprio', age=46)

## json output parser

In [28]:
parser = JsonOutputParser()

query="""Who is the actor who plays the male lead in the movie Titanic?"""

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": query})

{'movie': 'Titanic', 'male_lead_actor': 'Leonardo DiCaprio'}

In [29]:
query="""Who is the actor who plays the male lead in the movie Titanic? 
Tell me the age and name of the actor"""
chain.invoke({"query": query})

{'actor': 'Leonardo DiCaprio', 'age': 46}

In [26]:
query="""Who is the actor who plays the male lead in the movie Titanic? 
Tell me the age and name of the actor"""

parser = JsonOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": query})

{'name': 'Leonardo DiCaprio', 'age': 46}

## structured output parser

In [82]:
response_schemas = [
    ResponseSchema(name="name", description="name of the actor"),
    ResponseSchema(
        name="age",
        description="age of the actor",
    ),
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [84]:
query="""Who is the actor who plays the male lead in the movie Titanic? 
Tell me the age and name of the actor"""
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | model | parser
chain.invoke({"query": query})

{'name': 'Leonardo DiCaprio', 'age': '46'}

## output fixing parser

In [48]:
response="{'name': 'anna', 'age': '24'}"
pydantic_parser = PydanticOutputParser(pydantic_object=Actor)
pydantic_parser.parse(response)

OutputParserException: Invalid json output: {'name': 'anna', 'age': '24'}

In [49]:
response="{'name': 'anna', 'age': '24'}"
fixing_parser = OutputFixingParser.from_llm(parser=pydantic_parser, llm=model)
fixing_parser.parse(response)

Actor(name='Anna', age=24)

In [45]:
query="""Tell me just name of actor who plays the male lead in the movie Titanic"""

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
)

chain=prompt|model

response=chain.invoke(query).content

try:
    parsed_output = pydantic_parser.parse(response)
except Exception as e:
    print('output fixing parser is working')
    fixed_output = fixing_parser.parse(response)
    print(f'fixed_output:{fixed_output}')

output fixing parser is working
fixed_output:name='Leonardo DiCaprio' age=46


## Retry parser

In [72]:
class Stock(BaseModel):
    name: str=Field(description="company name")
    site: str=Field(description="site where i can find more stock price")
pydantic_parser = PydanticOutputParser(pydantic_object=Stock)

query="Tell me stock price of microsoft."
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
)
prompt= prompt.format_prompt(query=query)

response='{"site": "Yahoo Finance"}'

In [75]:
#output fixing parser
fixing_parser = OutputFixingParser.from_llm(parser=pydantic_parser, llm=model)
fixing_parser.parse(response)

Stock(name='ABC Company', site='https://www.abccompany.com/stock-price')

In [74]:
#retry parser
retry_parser = RetryOutputParser.from_llm(parser=pydantic_parser, llm=model)
retry_parser.parse_with_prompt(response, prompt)

Stock(name='Microsoft', site='Yahoo Finance')

## custom output parser

In [87]:
def parse(ai_message: AIMessage)->List[str]:
    return ai_message.content.split(" ")

chain=model|parse
chain.invoke("Who is the actor who plays the male lead in the movie Titanic?")

['Leonardo',
 'DiCaprio',
 'played',
 'the',
 'male',
 'lead,',
 'Jack',
 'Dawson,',
 'in',
 'the',
 'movie',
 'Titanic.']